In [11]:
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import StratifiedKFold


In [2]:
# Check if TensorFlow can access any GPU devices
gpu_devices = tf.config.list_physical_devices('GPU')

if len(gpu_devices) == 0:
    print("No GPU devices found.")
else:
    print(f"GPU devices found: {gpu_devices}")

# Check which GPU device is currently being used (if any)
current_gpu_device = tf.test.gpu_device_name()
if current_gpu_device:
    print(f"Current GPU device: {current_gpu_device}")
else:
    print("No GPU device is currently being used.")

# Check if GPU acceleration is available for TensorFlow
if tf.test.is_built_with_cuda():
    print("TensorFlow is built with CUDA support (GPU acceleration).")
else:
    print("TensorFlow is not built with CUDA support (GPU acceleration).")

    
print(tf.config.list_physical_devices('GPU'))

GPU devices found: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Current GPU device: /device:GPU:0
TensorFlow is built with CUDA support (GPU acceleration).
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-10-09 15:36:21.783562: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-09 15:36:21.838010: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-09 15:36:21.838159: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4" 

In [4]:
df = pd.read_csv('sk_dataset/slovak_data4_preprocessed.csv')
df.head()

,tweet,label
0,Marek Koolily hovorí s jasnejšími a faktami.,0
1,Vyhýbal som sa parlamentným voľbám už veľmi dlho,0
2,Tak,0
3,Nech vás Boh žehná za prezidenta Zuzana Capputová,0
4,Je to tiež osoba na správnom mieste.Ľudia nás ...,0


In [5]:
#Split the train data according to their class
off_df = df[df['label'] == 1]
non_off_df = df[df['label'] == 0]
off_df.shape, non_off_df.shape

((13158, 2), (13150, 2))

In [12]:
# Define the  metric result lists 
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
cv = 5  # Number of cross-validation folds
kf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

In [7]:
# Load the bert encoder model
bert_preprocess = hub.load(preprocess_url)
bert_encoder = hub.KerasLayer(encoder_url)

2023-10-09 15:36:33.861333: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-09 15:36:33.861534: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-09 15:36:33.861612: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [13]:
# Defining the model
def bert_based_model(dropout_rate=0.1):
    # Bert layers
    input_text = tf.keras.layers.Input(shape=(), dtype=tf.string, name='inputs')

    # Create a KerasLayer for tokenization
    tokenize = hub.KerasLayer(bert_preprocess.tokenize)
    tokenized_inputs = tokenize(input_text)

    tokenized_inputs.shape

    # Pack input sequences for the Transformer encoder (if needed)
    seq_length = 128  
    bert_pack_inputs = hub.KerasLayer(
        bert_preprocess.bert_pack_inputs,
        arguments=dict(seq_length=seq_length))  # Optional argument.

    encoder_inputs = bert_pack_inputs([tokenized_inputs])
    outputs = bert_encoder(encoder_inputs)

    # Neural network layers
    #layer = tf.keras.layers.BatchNormalization()(layer)
    layer = tf.keras.layers.Dropout(dropout_rate)(outputs["pooled_output"])  # Dropout layer
    layer = tf.keras.layers.Dense(128, activation="relu")(layer)
    #layer = tf.keras.layers.BatchNormalization()(layer)

    layer = tf.keras.layers.Dropout(dropout_rate)(layer)  # Dropout layer
    layer = tf.keras.layers.Dense(64, activation="relu")(layer)

    layer = tf.keras.layers.Dropout(dropout_rate)(layer)
    layer = tf.keras.layers.Dense(32, activation="relu")(layer)
    #layer = tf.keras.layers.BatchNormalization()(layer)


    layer = tf.keras.layers.Dropout(dropout_rate)(layer)

    # Output layer for binary classification
    output = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(layer)

    # Use inputs and outputs to construct a final model
    model = tf.keras.Model(inputs=[input_text ], outputs = [output])
    return model



In [14]:
import os
X = df['tweet'].values  
y = df['label'].values  

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Compile the model
    model = bert_based_model()

    # Compile and train the model
    checkpoint_dir = "checkpoints/checkpoints_mul/sk_checkpoint4"

    # creates the folder if not exist
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir, mode = 0o777)

    checkpoint_file_format = os.path.join(checkpoint_dir, "model_epoch_{epoch:02d}--loss-{loss:.4f}.h5")

    # Create a ModelCheckpoint callback
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_file_format,
        save_weights_only=True,  # Save only the model's weights
        save_best_only=True,     # Save the best one
        monitor="loss",          # Monitor loss
        mode="auto",             # Auto-detect whether to minimize or maximize the monitored metric
        verbose=1)               # Show progress during training

    #Compile the model
    METRICS = [
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=METRICS)
                 

    model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.1, callbacks=[model_checkpoint_callback])

    # Evaluate the model on the test set
    eval_metrics = model.evaluate(X_test, y_test)

    accuracy_scores.append(eval_metrics[1])  # Accuracy
    precision_scores.append(eval_metrics[2])  # Precision
    recall_scores.append(eval_metrics[3])     # Recall
    f1_scores.append(2 * (eval_metrics[2] * eval_metrics[3]) / (eval_metrics[2] + eval_metrics[3]))  # F1-score

    # Generate and print the classification report
    y_pred = model.predict(X_test)
    y_pred = y_pred.flatten()
    y_pred = np.where(y_pred > 0.5, 1, 0)

    classification_rep = classification_report(y_test, y_pred, target_names=['Class 0', 'Class 1'])
    print(f"Classification Report for Fold {len(accuracy_scores)}:\n", classification_rep)



Epoch 1/5


2023-10-09 15:54:55.858021: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb2604b1080 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-09 15:54:55.858045: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2023-10-09 15:54:55.862802: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-09 15:54:55.876444: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2023-10-09 15:54:55.934096: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


592/592 [==============================] - ETA: 0s - loss: 0.6022 - accuracy: 0.6507 - precision: 0.6110 - recall: 0.5897
Epoch 1: loss improved from inf to 0.60216, saving model to checkpoints/checkpoints_mul/sk_checkpoint4/model_epoch_01--loss-0.6022.h5
592/592 [==============================] - 234s 385ms/step - loss: 0.6022 - accuracy: 0.6507 - precision: 0.6110 - recall: 0.5897 - val_loss: 0.5668 - val_accuracy: 0.8171 - val_precision: 1.0000 - val_recall: 0.8171
Epoch 2/5
592/592 [==============================] - ETA: 0s - loss: 0.5468 - accuracy: 0.6902 - precision: 0.6221 - recall: 0.7727
Epoch 2: loss improved from 0.60216 to 0.54677, saving model to checkpoints/checkpoints_mul/sk_checkpoint4/model_epoch_02--loss-0.5468.h5
592/592 [==============================] - 229s 387ms/step - loss: 0.5468 - accuracy: 0.6902 - precision: 0.6221 - recall: 0.7727 - val_loss: 0.5722 - val_accuracy: 0.8665 - val_precision: 1.0000 - val_recall: 0.8665
Epoch 3/5
592/592 [=====================

In [15]:
print(f'Mean Accuracy: {np.mean(accuracy_scores)}')
print(f'Standard Deviation Accuracy: {np.std(accuracy_scores)}')

print(f'Mean Precision: {np.mean(precision_scores)}')
print(f'Standard Deviation Precision: {np.std(precision_scores)}')

print(f'Mean Recall: {np.mean(recall_scores)}')
print(f'Standard Deviation Recall: {np.std(recall_scores)}')

print(f'Mean F1-Score: {np.mean(f1_scores)}')
print(f'Standard Deviation F1-Score: {np.std(f1_scores)}')

Mean Accuracy: 0.7285237073898315
Standard Deviation Accuracy: 0.0050702746010004305
Mean Precision: 0.6791648983955383
Standard Deviation Precision: 0.006355212046746722
Mean Recall: 0.8675374507904052
Standard Deviation Recall: 0.03337926529584058
Mean F1-Score: 0.7614636281781952
Standard Deviation F1-Score: 0.010124273970846417


In [16]:
slovak_sentences = [
    "Dnes je krásny slnečný deň.",
    "Včera som čítal knihu v parku.",
    "Obľubujem dlhé prechádzky v prírode.",
    "toto je hlúpa lekcia",
    "nenávidím ťa vidieť plakať"
]
model.predict(slovak_sentences)

1/1 [==============================] - 0s 27ms/step


array([[0.02600755],
       [0.00612317],
       [0.00174148],
       [0.65219176],
       [0.523427  ]], dtype=float32)